In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from account.Binance import Binance
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import cvxpy as cp
from utils.logging import get_logger
from utils.data_helper import *
from utils.db import *
from strategy_v3.Strategy import ExchangeArbitrageStrategy

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.4f}".format
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Expermental Arbitrage strategy in Binance

1. Get bid/ask price for all active currency pairs in Binance and presents in a matrix $Q$

2. Transform the price to negative log price (Given converison from currency A->B->C = log(p1*p2) ~= log(p1) + log(p2))

3. Solve the optimization follow classic Traveling Salesmen Problem (TSP), but removing the constraints that all nodes needs to be visited once

- We want to find a closed loop where the sum of path values are negative

- Input $X$ is the nxn binary matrix (n is number of assets), 1 represents trade from currency x -> y

- Minimize $X$ dot $Q$

Reference: https://nbviewer.org/github/rcroessmann/sharing_public/blob/master/arbitrage_identification.ipynb


In [2]:
strategy = ExchangeArbitrageStrategy(zero_fees=False, max_trades=5, bid_ask_min=2, min_pnl=0)
strategy.set_strategy_id("qa")
strategy.load_data()
strategy.optimize()
#df_trades = strategy.df_trades
#df_trades

2025-02-03 02:53:48,104 - 739184 - INFO - trade_size: 50
2025-02-03 02:53:48,111 - 739184 - INFO - bid_ask_min: 2 ($100)
2025-02-03 02:53:48,112 - 739184 - INFO - zero_fees: False
2025-02-03 02:53:48,130 - 739184 - INFO - max_trades: 5
2025-02-03 02:53:48,137 - 739184 - INFO - min_pnl: 0bps
2025-02-03 02:53:48,142 - 739184 - INFO - is_execute: False
2025-02-03 02:53:48,330 - qa - INFO - {'USDT': 50, 'ETH': 0.0005149006851834908, 'BTC': 0.017065108508492452}
2025-02-03 02:53:48,434 - qa - INFO - Removed 104 out of 1338 symbols given both bid/ask are non-tradable <$100
2025-02-03 02:53:48,512 - qa - INFO - 346 bid/ask quotes are skipped given size <$100.


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Feb 03 02:53:48 AM: Your problem has 152881 variables, 5 constraints, and 0 parameters.
(CVXPY) Feb 03 02:53:48 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 03 02:53:48 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 03 02:53:48 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 03 02:53:48 AM: Compiling problem (target solver=SCIPY).
(CVXPY) Feb 03 02:53:48 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

2025-02-03 02:53:54,916 - qa - INFO - CVXPY - Status: optimal
2025-02-03 02:53:54,917 - qa - INFO - CVXPY - Optimal value: 0.0
2025-02-03 02:53:54,924 - qa - INFO - Total PNL: 0.00bps
2025-02-03 02:53:54,934 - qa - INFO - No optimal trades found, end here.


False

In [190]:
# duck("binance_arb_zero_fees", read_only=False).query(f"delete from trades where price_time::date <= '{datetime.today() - BDay(0):%Y-%m-%d}'")

,Count
0,27563


# Summary of strategy

###  breakdown the pnl slippage by
1. price slippage => difference between theoretical price and fill price
    - This can be improved by looking into order books and derive a fair market price based on trade size
    - Or just remove ccys with bid/ask notional smaller than x% of trade size

2. qty slippage => ccy might not be fully executed when baseAsset is not from_asset
    - Could mitigate by increase the trade size

3. commission
    - Limit the number of trades in optimization

In [4]:
df_trade_ccy_price = pd.DataFrame({
    'trade_ccy': ['USDT', 'ETH', 'BTC'], 
    'price': [1, float(Binance().client.get_avg_price(symbol="ETHUSDT")['price']), float(Binance().client.get_avg_price(symbol="BTCUSDT")['price'])]}
)

In [5]:
db = duck("binance_arb_zero_fees")
db.query("""--sql
    select
        price_time::date as "date"
        , epoch(price_time)::int as id
        , strftime(price_time, '%Y-%m-%d %H:%M:%S') as time
        , 10000 * (product(mkt_price_w_fee) - 1) as pnl_net
        , 10000 * (product(mkt_price) - 1) as pnl_gross
        , pnl_gross - pnl_net as fees
        , count(1) as num_trades
        , string_agg(from_asset, ', ') as ccy
        , bool_or(from_asset in ('USDT', 'ETH', 'BTC')) as tradable                       
    from trades         
    where 
        1=1                
    group by price_time      
    order by price_time desc    
    limit 20
""")

,date,id,time,pnl_net,pnl_gross,fees,num_trades,ccy,tradable
0,2025-02-02,1738500913,2025-02-02 12:55:13,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
1,2025-02-02,1738500854,2025-02-02 12:54:13,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
2,2025-02-02,1738500794,2025-02-02 12:53:13,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
3,2025-02-02,1738500734,2025-02-02 12:52:13,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
4,2025-02-02,1738500674,2025-02-02 12:51:14,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
5,2025-02-02,1738500614,2025-02-02 12:50:13,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
6,2025-02-02,1738500553,2025-02-02 12:49:13,3.2333,22.7678,19.5345,3,"PLN, USDT, USDC",True
7,2025-02-02,1738498393,2025-02-02 12:13:13,1.7930,31.8578,30.0648,3,"BNB, USDT, UAH",True
8,2025-02-02,1738498334,2025-02-02 12:12:14,4.8361,34.4076,29.5715,4,"BNB, USDC, USDT, UAH",True
9,2025-02-02,1738498214,2025-02-02 12:10:13,7.1506,37.2318,30.0812,3,"BNB, USDT, UAH",True


In [7]:
db = duck("binance_arb")
db.query("""--sql
    select
        price_time::date as "date"
        , epoch(price_time)::int as id
        , strftime(price_time, '%Y-%m-%d %H:%M:%S') as time
        , 10000 * (product(mkt_price_w_fee) - 1) as pnl_net
        , 10000 * (product(mkt_price) - 1) as pnl_gross
        , pnl_gross - pnl_net as fees
        , count(1) as num_trades
        , string_agg(from_asset, ', ') as ccy
        , bool_or(from_asset in ('USDT', 'ETH', 'BTC')) as tradable                       
    from trades         
    where 
        1=1                
    group by price_time       
    order by price_time desc    
    limit 20
""")

,date,id,time,pnl_net,pnl_gross,fees,num_trades,ccy,tradable
0,2025-01-29,1738164126,2025-01-29 15:22:05,0.0034,0.0034,0.0000,3,"FDUSD, USDT, USDC",True
1,2025-01-28,1738101186,2025-01-28 21:53:05,-0.0007,-0.0007,0.0000,3,"FDUSD, USDC, USDT",True
2,2025-01-28,1738067766,2025-01-28 12:36:06,0.0003,0.0003,0.0000,3,"FDUSD, USDC, USDT",True
3,2025-01-28,1738067165,2025-01-28 12:26:04,0.0003,0.0003,0.0000,3,"FDUSD, USDC, USDT",True
4,2025-01-28,1738065005,2025-01-28 11:50:05,-0.0001,-0.0001,0.0000,3,"FDUSD, USDC, USDT",True
5,2025-01-28,1738055046,2025-01-28 09:04:05,10.9597,51.1041,40.1444,4,"FDUSD, SOL, USDT, PENGU",True
6,2025-01-28,1738030805,2025-01-28 02:20:04,0.0040,0.0040,0.0000,3,"FDUSD, USDT, USDC",True
7,2025-01-27,1738015146,2025-01-27 21:59:06,-0.0003,-0.0003,0.0000,3,"FDUSD, USDC, USDT",True
8,2025-01-26,1737921186,2025-01-26 19:53:05,-0.0001,-0.0001,0.0000,3,"FDUSD, USDC, USDT",True
9,2025-01-26,1737891006,2025-01-26 11:30:05,-0.0001,-0.0001,0.0000,3,"FDUSD, USDC, USDT",True


In [28]:
db = duck("binance_arb")

# get raw orders
df_orders = db.query("""--sql
    select 
        *            
        , epoch(price_time)::int as id
        , price_time::date as "date"        
        , strftime(price_time, '%Y-%m-%d %H:%M:%S') as time
        , string_agg(distinct from_asset, ', ') over (partition by id, "group" order by "order" rows between unbounded preceding and unbounded following) as ccys        
        , first(from_asset) over (partition by price_time, "group" order by "order" rows between unbounded preceding and unbounded following) as trade_ccy
        , case when to_asset = quoteAsset then fill_price else 1/fill_price end as fill_price_adj                             
    from orders  
    where 1 = 1    
""")

# aggregate Orders to trade level
df_fills = duckdb.query("""--sql
    select
        "date"
        , "time"
        , "id"
        , ccys
        , trade_ccy
        , "order"
        , symbol
        , from_asset
        , to_asset                            
        , avg(mkt_price) as theo_px
        , avg(mkt_price_w_fee) as theo_px_w_fee
        , sum(fill_price_adj * fill_qty) / sum(fill_qty) as fill_px        
        , sum(from_asset_qty) as from_qty
        , sum(to_asset_qty) as to_qty                
        , sum(to_asset_comms_qty) as comms_qty
        , ifnull(lag(sum(to_asset_qty)) over (partition by id order by "order"), from_qty) as start_qty
        , start_qty - from_qty as residual_qty
        , 10000 * (fill_px / theo_px - 1) as px_slippage
        , 10000 * (-residual_qty / start_qty) as qty_slippage
        , 10000 * (-comms_qty / (comms_qty + to_qty)) as comms      
        , 10000 * ((1+px_slippage/10000) * (1+qty_slippage/10000) * (1+comms/10000) - 1) as slippage                                                            
        , count(1) as fills        
    from df_orders    
    group by "date", "time", "id", ccys, trade_ccy, "order", from_asset, to_asset, symbol
    order by id, "order"
""").to_df()

In [29]:
df_fills_agg = duckdb.query("""--sql
    select
        "date"
        , id
        , "time"
        , ccys
        , t1.trade_ccy
        , first(from_asset) = last(to_asset) as valid
        , sum(fills)::int as fills
        , count(1) as trades        
        , first(from_qty) as from_qty
        , last(to_qty) as to_qty
        , last(to_qty) - first(from_qty) as net_qty
        , net_qty * avg(price) as net_pnl_usd 
        , 10000 * net_qty / first(from_qty) as realized_pnl
        , 10000 * (product(theo_px) - 1) as theo_gross_pnl
        , 10000 * (product(theo_px_w_fee) - 1) as theo_net_pnl                            
        , 10000 * (product(1+px_slippage/10000)-1) as px_slippage
        , 10000 * (product(1+qty_slippage/10000)-1) as qty_slippage
        , 10000 * (product(1+comms/10000)-1) as comms                 

    from df_fills t1
    left join df_trade_ccy_price t2 on t1.trade_ccy = t2.trade_ccy
    group by "date", id, "time", ccys, t1.trade_ccy 
    order by time desc
""").to_df()

df_fills_agg

,date,id,time,ccys,trade_ccy,valid,fills,trades,from_qty,to_qty,net_qty,net_pnl_usd,realized_pnl,theo_gross_pnl,theo_net_pnl,px_slippage,qty_slippage,comms
0,2025-01-21,1737461405,2025-01-21 12:10:05,"USDT, IOTX, JPY, BTC",USDT,True,7,4,49.9814,50.1474,0.1659,0.1659,33.1968,97.0209,56.6943,-6.0647,-17.3306,-39.9197
1,2025-01-21,1737451984,2025-01-21 09:33:04,"USDT, XRP, MXN",USDT,True,5,3,49.4000,46.9530,-2.4470,-2.4470,-495.3445,84.9450,54.7214,-316.3797,-238.2282,-29.9700
2,2025-01-21,1737431705,2025-01-21 03:55:05,"USDT, USDC, TRUMP",USDT,True,5,3,49.0539,48.8510,-0.2029,-0.2029,-41.3712,178.3774,158.5394,-185.8366,-11.1613,-19.4905
3,2025-01-21,1737427205,2025-01-21 02:40:05,"ETH, IOTX, JPY",ETH,True,5,3,0.0154,0.0155,0.0001,0.3060,61.6712,89.4338,59.1960,9.3859,-6.9160,-29.9700
4,2025-01-21,1737423786,2025-01-21 01:43:06,"USDT, EURI, BTC, HIVE",USDT,True,4,4,49.9248,48.2046,-1.7202,-1.7202,-344.5490,103.9847,73.7024,-137.8807,-281.1878,-29.9700
5,2025-01-21,1737418386,2025-01-21 00:13:05,"BTC, USDC, PEPE, JPY",BTC,True,5,4,0.0005,0.0005,-0.0000,-0.1601,-29.2303,99.0738,59.7451,-74.3081,-14.2697,-38.9182
6,2025-01-20,1737387365,2025-01-20 15:36:04,"BTC, BRL, PEPE, JPY",BTC,True,10,4,0.0005,0.0005,-0.0000,-0.1601,-29.2304,90.5819,50.2800,-63.8951,-15.4549,-39.7998
7,2025-01-20,1737387246,2025-01-20 15:34:05,"BTC, USDC, PEPE, JPY",BTC,True,9,4,0.0005,0.0005,0.0000,0.1559,28.4613,108.2882,68.9236,-33.8029,-6.4423,-38.9048
8,2025-01-20,1737370144,2025-01-20 10:49:04,"USDT, RON, EGLD",USDT,True,7,3,50.0000,48.7452,-1.2548,-1.2548,-250.9590,126.2664,95.9174,-270.7469,-74.8605,-29.9700
9,2025-01-20,1737352867,2025-01-20 06:01:06,"USDT, SNT, BTC, EURI",USDT,True,9,4,49.9727,48.5708,-1.4019,-1.4019,-280.5365,90.1658,59.9257,-215.7623,-125.2798,-30.0670


# Trade the arbitrage pair(s)
- Trade the pairs with highest pnl
- The arbitrage loop starts with one existing currency in current portfolios

- commission are included in quote quantity?

In [97]:
trade_currency = {
    'USDT': 50,
    'ETH':  0.01539456264,
    'BTC': 0.00052979531,
}

In [254]:
client = Binance().client
balance = client.get_account()
balance = pd.DataFrame(balance['balances'])
balance['free'] = balance['free'].astype(float)
balance['locked'] = balance['locked'].astype(float)
balance = balance[balance['free'] > 0]
balance

,asset,free,locked
0,BTC,0.0842,0.0233
2,ETH,1.8625,0.0000
4,BNB,0.0000,0.0000
7,SNT,0.8130,0.0000
11,USDT,"6,900.0469","1,361.2542"
22,TRX,0.0821,0.0000
57,VIB,0.3160,0.0000
61,XRP,2.7227,0.0000
86,ADA,0.0734,0.0000
88,XLM,99.9000,0.0000
